<a href="https://colab.research.google.com/github/ayssag/7DaysOfCodePandas/blob/main/_7DaysOfCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando dados

In [1]:
%%capture
!pip install pandas

In [2]:
repo = 'https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas.git'
!git clone {repo}

Cloning into '7_Days_of_Code_Alura-Python-Pandas'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 282 (delta 29), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (282/282), 82.88 MiB | 7.45 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Updating files: 100% (33/33), done.


In [58]:
import os
import pandas as pd

path = '7_Days_of_Code_Alura-Python-Pandas/Dia_1-Importando_dados/Datasets'

files = os.listdir(f'{path}/dados_emprestimos')
emprestimos = pd.DataFrame()

for file in files:
  add_emprestimos = pd.read_csv(f'{path}/dados_emprestimos/{file}')
  emprestimos = pd.concat([emprestimos, add_emprestimos], ignore_index=True)


In [59]:
files = os.listdir(path)
exemplares =  pd.read_parquet(f'{path}/dados_exemplares.parquet')

In [111]:
dataset = emprestimos.merge(exemplares, on='codigo_barras', how='left')

# Limpeza de dados

In [112]:
dataset = dataset.drop_duplicates()

In [113]:
dataset = dataset.drop(columns='registro_sistema')

In [114]:
columns_with_nan = dataset.isnull().any()
# data_renovacao, data_devolucao, matricula_ou_siape, id_exemplar, colecao, biblioteca, status_material, localizacao, registro_sistema

for col, _ in columns_with_nan.items():
  if col:
    dataset[col] = dataset[col].replace(float('nan'), 'unknown')


In [115]:
def convert_to_string(value):
  return str(value)

dataset['matricula_ou_siape'] = list(map(convert_to_string, dataset['matricula_ou_siape']))

In [116]:
columns_with_na = dataset.isna().any()
# localizacao
for col, _ in columns_with_na.items():
  if col:
    dataset[col] = dataset[col].replace(pd.NA, 'unknown')


In [117]:
def convert_cdu(value):
  if value != 'unknown':
    match value:
      case value if float(value) >= 0 and float(value) <= 99:
        return 'Generalidades. Ciência e conhecimento.'
      case value if float(value) >= 100 and float(value) <= 199:
        return 'Filosofia e psicologia.'
      case value if float(value) >= 200 and float(value) <= 299:
        return 'Religião.'
      case value if float(value) >= 300 and float(value) <= 399:
        return 'Ciências sociais.'
      case value if float(value) >= 400 and float(value) <= 499:
        return 'Classe vaga. Provisoriamente não ocupada.'
      case value if float(value) >= 500 and float(value) <= 599:
        return 'Matemática e ciências naturais.'
      case value if float(value) >= 600 and float(value) <= 699:
        return 'Ciências aplicadas.'
      case value if float(value) >= 700 and float(value) <= 799:
        return 'Belas artes.'
      case value if float(value) >= 800 and float(value) <= 899:
        return 'Linguagem. Língua. Linguística.'
      case value if float(value) >= 900 and float(value) <= 999:
        return 'Geografia. Biografia. História.'
  else:
    return value

dataset['localizacao_cd'] = list(map(convert_cdu, dataset['localizacao']))

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,localizacao_cd
0,1535182,E002010,unknown,2013/07/01 07:47:13.765000000,2013/08/06 20:28:21.000000000,2012320365.0,ALUNO MÉDIO/TÉCNICO,unknown,unknown,unknown,unknown,unknown,unknown
1,1535183,2010049079,2013/07/16 17:25:10.615000000,2013/07/01 07:47:23.170000000,2013/07/18 09:50:26.000000000,2009007190.0,ALUNO DE GRADUAÇÃO,939119.0,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,668.0,Ciências aplicadas.
2,1535184,2011013154,2013/07/16 21:03:55.928000000,2013/07/01 07:54:01.549000000,2013/07/30 08:39:22.000000000,2011018150.0,ALUNO DE GRADUAÇÃO,923040.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,660.0,Ciências aplicadas.
3,1535185,2011001485,2013/07/16 21:03:55.802000000,2013/07/01 07:54:01.611000000,2013/07/17 08:56:58.000000000,2011018150.0,ALUNO DE GRADUAÇÃO,982496.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,310.0,Ciências sociais.
4,1535186,2012038972,2013/07/16 17:51:37.835000000,2013/07/01 08:23:51.535000000,2013/07/31 13:51:11.000000000,2011092659.0,ALUNO DE GRADUAÇÃO,1025496.0,Acervo Circulante,Biblioteca Setorial da Faculdade de Ciências d...,REGULAR,610.0,Ciências aplicadas.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261811,1149744,L133454,unknown,2011/12/30 17:38:05.838000000,2012/01/30 13:38:38.000000000,2010210883.0,ALUNO DE PÓS-GRADUAÇÃO,36689.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,654.0,Ciências aplicadas.
2261812,1149745,2008053028,2012/01/30 09:07:40.289000000,2011/12/30 17:42:51.232000000,2012/02/29 13:39:55.000000000,1755267.0,DOCENTE,159775.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,635.0,Ciências aplicadas.
2261813,1149746,2008053053,2012/01/30 09:06:19.941000000,2011/12/30 17:42:51.274000000,2012/02/13 14:28:37.000000000,1755267.0,DOCENTE,217526.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,662.0,Ciências aplicadas.
2261814,1149747,2010066523,2012/01/15 19:06:56.012000000,2011/12/30 17:46:47.021000000,2012/01/19 11:30:36.000000000,2011056198.0,ALUNO DE GRADUAÇÃO,813168.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,604.0,Ciências aplicadas.
